In [ ]:
library(tidyverse)

## Constants

In [ ]:
data_dir <- "../../../../../mnt/d"
analysis_dir <- paste0(data_dir, "/", "unified_TCGA_GTEx/analysis")
master_list <- "matrisome/matrisome_hs_masterlist.tsv"
DEG_list <- "unified_TCGA_GTEx/analysis/unified_cervical_data_DGE_padj0.05.tsv"

In [ ]:
DEG_df <- read_tsv(paste0(data_dir, "/", DEG_list))
# some cells have double quotes inside of them
matrisome_df <- read_tsv(paste0(data_dir, "/", master_list), quote = "")
colnames(matrisome_df) <- map(sub(" ", "_", colnames(matrisome_df)), tolower)
matrisome_df <- select(matrisome_df, gene_symbol, everything()) %>%
    dplyr::filter(division != "Retired")

In [ ]:
# Importand for next step that this is FALSE
any(is.na(matrisome_df$division))

In [ ]:
joined_df <- left_join(DEG_df, matrisome_df, by = c("geneID" = "gene_symbol")) %>%
    mutate(in_matrisome = !is.na(division)) %>%    # We know a priori that division is never NA
    select(geneID:padj, in_matrisome, everything())

In [ ]:
nrow(DEG_df) == nrow(joined_df)    # This must be true since we want to retain all DEGs
head(joined_df)

In [ ]:
category_counts_df <- joined_df %>% count(category) %>%
    mutate(n_genes = n) %>%
    select(-n) %>%
    filter(!is.na(category))

category_counts_df

In [ ]:
write_tsv(joined_df, paste0(analysis_dir, "/", "unified_cervical_DGE_matrisome_left_join.tsv"))
write_tsv(category_counts_df, paste0(analysis_dir, "/", "unified_cervical_DGE_matrisome_category_counts.tsv"))